## **III. Merge csv and API**

### What the code does:
* This code merges the datasets from TidyTuesday and the Board Game Atlas API and creates the workfile

**Import libraries**

In [1]:
import pandas as pd
import requests
import json
import time
import math

### Technical notes:
* The full list of codes in this workbook run for about 15 minutes, 10 of which is the fuzzy merge.
* For the fuzzy merge rapidfuzz module is required.


**Load in datasets**

In [4]:
# Import dataset - Tidytuesday
board_games = pd.read_csv("1loading_output.csv", sep=',')
board_games.columns = [str(col) + '_base' for col in board_games.columns]
board_games.head()

,Unnamed: 0_base,game_id_base,description_base,max_players_base,max_playtime_base,min_age_base,min_players_base,min_playtime_base,name_base,playing_time_base,year_published_base,artist_base,category_base,designer_base,family_base,mechanic_base,publisher_base,average_rating_base,users_rated_base,name_lower_base
0,0,1,Die Macher is a game about seven sequential po...,5,240,14,3,240,Die Macher,240,1986,Marcus Gschwendtner,"Economic,Negotiation,Political",Karl-Heinz Schmiel,"Country: Germany,Valley Games Classic Line","Area Control / Area Influence,Auction/Bidding,...","Hans im Glück Verlags-GmbH,Moskito Spiele,Vall...",7.66508,4498,die macher
1,1,2,Dragonmaster is a trick-taking card game based...,4,30,12,3,30,Dragonmaster,30,1981,Bob Pepper,"Card Game,Fantasy","G. W. ""Jerry"" D'Arcey",Animals: Dragons,Trick-taking,"E.S. Lowe,Milton Bradley",6.60815,478,dragonmaster
2,2,3,"Part of the Knizia tile-laying trilogy, Samura...",4,60,10,2,30,Samurai,60,1998,Franz Vohwinkel,"Abstract Strategy,Medieval",Reiner Knizia,"Asian Theme,Country: Japan,Knizia tile-laying ...","Area Control / Area Influence,Hand Management,...","999 Games,ABACUSSPIELE,Astrel Games,Ceilikan J...",7.44119,12019,samurai
3,3,4,When you see the triangular box and the luxuri...,4,60,12,2,60,Tal der Könige,60,1992,NaN,Ancient,Christian Beierer,"Country: Egypt,Promotional Board Games","Action Point Allowance System,Area Control / A...",KOSMOS,6.60675,314,tal der könige
4,4,5,"In Acquire, each player strategically invests ...",6,90,12,3,90,Acquire,90,1964,"Scott Okumura,Peter Whitley",Economic,Sid Sackson,3M Bookshelf Series,"Hand Management,Stock Holding,Tile Placement","3M,Avalon Hill,Avalon Hill (Hasbro),Dujardin,G...",7.35830,15195,acquire


In [5]:
board_games_api = pd.read_csv("2loading_output.csv", sep=',')
board_games_api.columns = [str(col) + '_api' for col in board_games_api.columns]
board_games_api.head()

,Unnamed: 0_api,id_api,name_api,price_api,price_ca_api,price_uk_api,price_au_api,discount_api,year_published_api,min_players_api,...,average_learning_complexity_api,average_strategy_complexity_api,visits_api,plays_api,listing_clicks_api,is_historical_low_api,players_api,playtime_api,name_lower_api,index_api
0,0,o6knI5ct0u,1830: Railways & Robber Barons,52.99,74.99,50.09,89.95,0.24,1986.0,2.0,...,3.0,4.5,1908,244,0,False,2-7,180-360,1830: railways & robber barons,NaN
1,1,eSCVHuUFPs,1846: The Race for the Midwest,42.25,67.95,50.88,0.00,0.39,2005.0,3.0,...,4.0,4.0,1089,315,7,False,3-5,240,1846: the race for the midwest,NaN
2,2,uEPtE5OOOU,1862: Railway Mania in the Eastern Counties,57.99,79.95,0.00,127.99,0.27,2013.0,1.0,...,0.0,0.0,1073,71,0,False,1-8,300,1862: railway mania in the eastern counties,NaN
3,3,Krn8i8C0fI,18Chesapeake,0.00,0.00,0.00,0.00,0.00,2020.0,2.0,...,3.0,4.0,870,160,0,False,2-6,180,18chesapeake,NaN
4,4,eJuGpFzljd,1844 / 1854,75.99,112.95,0.00,0.00,0.20,2016.0,3.0,...,0.0,0.0,505,5,0,False,3-7,300,1844 / 1854,NaN


**Try simple join based on names**

In [6]:
# Merge datasets on name and year_published
united_df = pd.merge(board_games, 
                     board_games_api, 
                     how='inner', 
                     left_on=['name_lower_base', 'year_published_base'], 
                     right_on=['name_lower_api', 'year_published_api'])
len(united_df)

4768

### Improvements, approximate match:
* The two API calls already made good results, almost half the csv games were found.
* But maybe we can do better by approximate ("fuzzy") search.


**Import libraries**

In [7]:
# !pip install rapidfuzz -q # fuzzy search package, install takes about 2 minutes
# from rapidfuzz import fuzz # main score package, not required
from rapidfuzz.process import extractOne, extract, extract_iter # get best match, get all matches, iterated
from rapidfuzz.string_metric import levenshtein, normalized_levenshtein # more complex match scores
# from rapidfuzz.fuzz import ratio #simple ratio, not required

In [8]:
board_games['name_lower_base'][0]

'die macher'

In [9]:
# example of finding all matches above 85% threshold
# Results are not very good
for x in extract_iter(board_games['name_lower_base'][0],board_games_api['name_lower_api'],score_cutoff = 85):
    print(x)

('the goonies: never say die', 85.5, 389)
('the goonies: under the goondocks – a never say die expansion', 85.5, 409)
('zicke & zacke: ran an die federn', 85.5, 1058)
('bauer klaus und die maus', 85.5, 1070)
('star wars: the card game - join us or die force pack', 85.5, 2013)
('pokémon trading card game: dragon majesty damage-counter dice and coin-flip die', 85.5, 3238)
('pokemon: sun & moon lost thunder - 6 damage-counter dice and 1 coin-flip die', 85.5, 3333)
('forged in steel: forged steel d4 die', 85.5, 3343)
('rund um die porta-westfalica', 85.5, 3403)
('die macher', 100.0, 3549)
('cthulhu: death may die - season 2 expansion', 85.5, 4237)
('run, fight, or die', 85.5, 5312)
('die of the dead', 85.5, 6161)
('die holde isolde', 85.5, 6455)
('squirrel or die', 85.5, 6471)
("don't let it die", 85.5, 7124)
('die, vecna, die! (ad&d 2e)', 85.5, 7422)
('this war of mine: rubble die', 85.5, 8797)
('uruk ii: die entwicklung geht weiter', 85.5, 8937)
('die legenden von andor: die jagd', 85.5,

In [10]:
# Examples of best match above threshold.
# This is much better
for game in board_games['name_lower_base'][0:15]:
    best_find = extractOne(game,board_games_api['name_lower_api'],score_cutoff = 85)
    print(game)
    if not (best_find is None):
        print(best_find[0])
        print(best_find[1])
    else:
        print('No match')

die macher
die macher
100.0
dragonmaster
dragonmaster
100.0
samurai
samurai
100.0
tal der könige
tal der könige
100.0
acquire
acquire
100.0
mare mediterraneum
terra
90.0
cathedral
cathedral
100.0
lords of creation
1822: the railways of great britain
85.5
el caballero
el caballero
100.0
elfenland
elfenland
100.0
bohnanza
bohnanza
100.0
ra
ra
100.0
catan
catan
100.0
basari
basari board game
90.0
cosmic encounter
cosmic encounter
100.0


In [11]:
# runs for about 10 minutes at 20000 rows
# get best match with score for all board games from raw file if there is any match of at least 85% score.
import numpy as np


matches = []
scores = []

for game in board_games['name_lower_base']:
    best_find = extractOne(game,board_games_api['name_lower_api'],score_cutoff = 85)
    if not (best_find is None):
        matches.append(best_find[0])
        scores.append(best_find[1])
    else:
        matches.append(np.nan)
        scores.append(np.nan)


In [12]:
# Create data.frame of scores

aux_df = pd.DataFrame({'csv': pd.Series(board_games['name_lower_base'] ), 'match': matches , 'score': scores})
aux_df

,csv,match,score
0,die macher,die macher,100.0
1,dragonmaster,dragonmaster,100.0
2,samurai,samurai,100.0
3,tal der könige,tal der könige,100.0
4,acquire,acquire,100.0
...,...,...,...
10525,"silver bayonet: the first team in vietnam, 196...",1846: the race for the midwest,85.5
10526,codex: card-time strategy – core set,codex: card-time strategy - core set,100.0
10527,wind the film!,wind the film!,100.0
10528,robo rally (2016),robo rally,90.0


In [13]:
# Percentage of matches found
(len(aux_df) - sum(aux_df['match'].isnull()))/len(aux_df)

0.9362773029439696

### Results:
* Over 93% of games were found a match, which seems a bit too optimistic.
* Also, some of the matches with scores between 85-95% are not very good.
* However, from the same module we imported we can use levenshtein scores which are more precise and can be customized.


**Examples of lower levenshtein scores, setting threshold at 80% seems good.**

In [14]:
normalized_levenshtein('space beans', 'space base',weights=(1,1,1))

72.72727272727273

In [15]:
normalized_levenshtein('taki', 'tak',weights=(1,1,1))

75.0

**However, longer names seem to be more complex.**

In [17]:
normalized_levenshtein('battle of the bulge','battle of the bulge 1944')

79.16666666666666

In [18]:
# We can set up weights.

normalized_levenshtein('battle of the bulge','battle of the bulge 1944',weights=(1,1,1)) # by default it has weights 1-1-1

# Weights stand for: 1 penalty for 1 digit additional - 1 digit less - 1 digit substituted

79.16666666666666

In [19]:
# Weights can be changed.

normalized_levenshtein('battle of the bulge','battle of the bulge 1944',weights=(1,1,2))

# It seem that for longer names, substitution is a bigger issuer than less/more words.

88.37209302325581

### Rules:
* Some board games have really long names, where substitutions are the biggest issue, whereas additional words are not so much.
* Examples such as: "third edition" , "collectible" are not as good indicators of false match.
* So for longer names we penalize only substitutions heavily.
* As tested, we will use 80% levenshtein match as threshold for true match.


In [21]:
weighted_score = []
for game in list(range(0,10530)):
    if not (aux_df['match'][game] is np.nan):
        if len(aux_df['match'][game]) > 15:
            weighted_score.append(normalized_levenshtein(aux_df['csv'][game], aux_df['match'][game],weights=(1,1,2)))
        else:
            weighted_score.append(normalized_levenshtein(aux_df['csv'][game], aux_df['match'][game],weights=(1,1,1)))
    else:
        weighted_score.append(np.nan)

In [52]:
aux_df['weighted_score'] = weighted_score
aux_df['threshold'] = np.where(aux_df['weighted_score']>=80,True,False)
print(sum(aux_df['threshold'])) # number of matches above threshold

5576


### Conclusions:
* Using fuzzy search with relatively conservative scores netted us over 15% more matches.
* Finally: merge auxilary results table, keep only true matches.
* Based on true matches, merge to api table.
* Remove technical columns.


In [97]:
# Add results to table to merge based on it
united_df = board_games.join(aux_df[['match','threshold']])
# merge on index as same length since we found something for each game
united_df

,Unnamed: 0_base,game_id_base,description_base,max_players_base,max_playtime_base,min_age_base,min_players_base,min_playtime_base,name_base,playing_time_base,...,category_base,designer_base,family_base,mechanic_base,publisher_base,average_rating_base,users_rated_base,name_lower_base,match,threshold
0,0,1,Die Macher is a game about seven sequential po...,5,240,14,3,240,Die Macher,240,...,"Economic,Negotiation,Political",Karl-Heinz Schmiel,"Country: Germany,Valley Games Classic Line","Area Control / Area Influence,Auction/Bidding,...","Hans im Glück Verlags-GmbH,Moskito Spiele,Vall...",7.66508,4498,die macher,die macher,True
1,1,2,Dragonmaster is a trick-taking card game based...,4,30,12,3,30,Dragonmaster,30,...,"Card Game,Fantasy","G. W. ""Jerry"" D'Arcey",Animals: Dragons,Trick-taking,"E.S. Lowe,Milton Bradley",6.60815,478,dragonmaster,dragonmaster,True
2,2,3,"Part of the Knizia tile-laying trilogy, Samura...",4,60,10,2,30,Samurai,60,...,"Abstract Strategy,Medieval",Reiner Knizia,"Asian Theme,Country: Japan,Knizia tile-laying ...","Area Control / Area Influence,Hand Management,...","999 Games,ABACUSSPIELE,Astrel Games,Ceilikan J...",7.44119,12019,samurai,samurai,True
3,3,4,When you see the triangular box and the luxuri...,4,60,12,2,60,Tal der Könige,60,...,Ancient,Christian Beierer,"Country: Egypt,Promotional Board Games","Action Point Allowance System,Area Control / A...",KOSMOS,6.60675,314,tal der könige,tal der könige,True
4,4,5,"In Acquire, each player strategically invests ...",6,90,12,3,90,Acquire,90,...,Economic,Sid Sackson,3M Bookshelf Series,"Hand Management,Stock Holding,Tile Placement","3M,Avalon Hill,Avalon Hill (Hasbro),Dujardin,G...",7.35830,15195,acquire,acquire,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10525,10527,214996,Description from the publisher:&#10;&#10;Silve...,2,480,12,1,30,"Silver Bayonet: The First Team in Vietnam, 196...",480,...,"Vietnam War,Wargame","Gene Billingsley,Mitchell Land",GMT Operational System,Hex-and-Counter,GMT Games,8.35333,75,"silver bayonet: the first team in vietnam, 196...",1846: the race for the midwest,False
10526,10528,215437,"Codex: Card-Time Strategy is a customizable, n...",5,45,13,2,45,Codex: Card-Time Strategy – Core Set,45,...,"Card Game,Fantasy,Fighting,Video Game Theme",David Sirlin,Fantasy Strike,"Deck / Pool Building,Hand Management,Variable ...",Sirlin Games,8.08780,82,codex: card-time strategy – core set,codex: card-time strategy - core set,True
10527,10529,215471,Time to walk about town and take some pictures...,4,20,12,2,20,Wind the Film!,20,...,Card Game,Saashi,NaN,"Hand Management,Set Collection",Saashi & Saashi,7.28016,63,wind the film!,wind the film!,True
10528,10530,216201,The race is on for the robots of the Robo Rall...,6,120,12,2,20,Robo Rally (2016),120,...,"Miniatures,Racing,Science Fiction",Richard Garfield,Robots,"Action / Movement Programming,Grid Movement,Mo...","Avalon Hill (Hasbro),Hasbro",7.45871,341,robo rally (2016),robo rally,False


In [98]:
# Filter out only matches above threshold
united_df = united_df[united_df['threshold'] == True] 

In [99]:
# Merge datasets on name and match found
# no_duplicate_board_games_api = board_games_api[~board_games_api.duplicated(subset=['name_lower_api', 'year_published_api'])]

united_df = pd.merge(united_df, 
                     board_games_api, 
                     how='inner', 
                     left_on=['match'], 
                     right_on=['name_lower_api'])
print(len(united_df))
# Remove duplicates from multiple matches
united_df = united_df[~united_df.duplicated(subset=['name_lower_base', 'year_published_base'])]
len(united_df)

5630


5576

In [100]:
united_df.columns

Index(['Unnamed: 0_base', 'game_id_base', 'description_base',
       'max_players_base', 'max_playtime_base', 'min_age_base',
       'min_players_base', 'min_playtime_base', 'name_base',
       'playing_time_base', 'year_published_base', 'artist_base',
       'category_base', 'designer_base', 'family_base', 'mechanic_base',
       'publisher_base', 'average_rating_base', 'users_rated_base',
       'name_lower_base', 'match', 'threshold', 'Unnamed: 0_api', 'id_api',
       'name_api', 'price_api', 'price_ca_api', 'price_uk_api', 'price_au_api',
       'discount_api', 'year_published_api', 'min_players_api',
       'max_players_api', 'min_playtime_api', 'max_playtime_api',
       'min_age_api', 'comment_count_api', 'num_user_ratings_api',
       'average_user_rating_api', 'average_learning_complexity_api',
       'average_strategy_complexity_api', 'visits_api', 'plays_api',
       'listing_clicks_api', 'is_historical_low_api', 'players_api',
       'playtime_api', 'name_lower_api', 'inde

In [101]:
united_df['Unnamed: 0'] = united_df.index # Adding new index
# Move it first
cols = united_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
united_df = united_df[cols] 
united_df = united_df.drop(['match', 'threshold','index_api'], axis = 1) # Removing technical columns


In [104]:
united_df.columns

Index(['Unnamed: 0', 'Unnamed: 0_base', 'game_id_base', 'description_base',
       'max_players_base', 'max_playtime_base', 'min_age_base',
       'min_players_base', 'min_playtime_base', 'name_base',
       'playing_time_base', 'year_published_base', 'artist_base',
       'category_base', 'designer_base', 'family_base', 'mechanic_base',
       'publisher_base', 'average_rating_base', 'users_rated_base',
       'name_lower_base', 'Unnamed: 0_api', 'id_api', 'name_api', 'price_api',
       'price_ca_api', 'price_uk_api', 'price_au_api', 'discount_api',
       'year_published_api', 'min_players_api', 'max_players_api',
       'min_playtime_api', 'max_playtime_api', 'min_age_api',
       'comment_count_api', 'num_user_ratings_api', 'average_user_rating_api',
       'average_learning_complexity_api', 'average_strategy_complexity_api',
       'visits_api', 'plays_api', 'listing_clicks_api',
       'is_historical_low_api', 'players_api', 'playtime_api',
       'name_lower_api'],
      dtype=

In [105]:
# Save dataset 
united_df.to_csv("board_games_workfile.csv", sep=';' , quotechar='"')